# SNLP project

## Ingredient suggestion task 

**Objective:**

Produce some suggestions for substitution for certain ingredients, e,g, “vegan” and “pesto pasta” might give us “tofu”.

In [79]:
import pandas as pd
import numpy as np
from typing import List
import gensim
from gensim.models import Word2Vec
import spacy
import nltk
from nltk.corpus import stopwords
from nltk import download
from nltk.stem import WordNetLemmatizer
import time 

In [80]:
df = pd.read_csv("./data/cleaneddata.csv", index_col=0)

In [81]:
# df.head()

## 1. Text preprocessing

### 1.1 Drop column we don't need and renaming columns

In [82]:
df = df.drop(["healthScore", "pricePerServing", "readyInMinutes", "servings"], axis=1)


In [83]:
df = df.rename(columns={
    "glutenFree": "gluten-free", 
    "dairyFree": "dairy-free", 
    "veryHealthy":"very-healthy", 
    "veryPopular": "very-popular", 
})

### 1.2 Transform the TRUE classification labels to class name

In [84]:
classes = ['vegetarian', 
           'vegan', 
           'gluten-free', 
           'dairy-free', 
           'very-healthy', 
           'cheap', 
           'very-popular', 
           'sustainable']

In [85]:
for c in classes:
    df[c] = df[c].replace(to_replace=[True, False], value=[c, np.nan])

In [86]:
df.head()

,title,summary,instructions,ingredients,ingredient types,diets,vegetarian,vegan,gluten-free,dairy-free,very-healthy,cheap,very-popular,sustainable
0,orange fig teacake with caramel glaze,orange fig teacake with caramel glaze is a veg...,you will need a 9 springform pan or a cake ...,ap flour; baking powder; cardamom; eggs; fresh...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,vegetarian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,poached eggs on a bed of fried mushrooms and c...,poached eggs on a bed of fried mushrooms and c...,in a frying pan heat up oil then add mushroom...,bread; butter; eggs; eggs; mushrooms; oil; sal...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,vegetarian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pandan chiffon cake,for 26 cents per serving this recipe covers ...,preheat the oven to 170c blend the pandan le...,all purpose flour; bay leaves; coconut milk; c...,Ethnic Foods Produce Spices and Seasonings Bev...,dairy free; lacto ovo vegetarian,vegetarian,NaN,NaN,dairy-free,NaN,NaN,NaN,NaN
3,pork chop with honey mustard and apples,pork chop with honey mustard and apples might...,pre heat your oven to 200c 400f line a roa...,apples; dijon mustard; garlic cloves; honey; j...,Meat Spices and Seasonings Condiments Oil Vine...,gluten free; dairy free; paleolithic; primal,NaN,NaN,gluten-free,dairy-free,NaN,NaN,NaN,NaN
4,beet gnocchi with steak and brown butter sauce,the recipe beet gnocchi with steak and brown b...,cooking beets heat oven to 400 degrees wash be...,gnocchi; beets; olive oil; s p; goat cheese; r...,Produce Spices and Seasonings Meat Spices and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.3 Creating two copies of the diets and ingredients columns

The ingredients and diets are going to be important for this tasks. To make our model more flexible, we will expand the vocabularies in these categories. 

For diets, we want to include both `"gluten free"` and `"gluten-free"` in the corpus. For ingredients, we want to include both `"extra virgin olive oil"` and `"extra", "virgin", "olive", "oil"`. 

In [87]:
# df["ingredient split"] = df["ingredients"].str.replace("; ", " ")
df["diet split"] = df["diets"].str.replace(" ", "-").str.replace(";-", "; ")

In [88]:
df.head()

,title,summary,instructions,ingredients,ingredient types,diets,vegetarian,vegan,gluten-free,dairy-free,very-healthy,cheap,very-popular,sustainable,diet split
0,orange fig teacake with caramel glaze,orange fig teacake with caramel glaze is a veg...,you will need a 9 springform pan or a cake ...,ap flour; baking powder; cardamom; eggs; fresh...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,vegetarian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lacto-ovo-vegetarian
1,poached eggs on a bed of fried mushrooms and c...,poached eggs on a bed of fried mushrooms and c...,in a frying pan heat up oil then add mushroom...,bread; butter; eggs; eggs; mushrooms; oil; sal...,Beverages Milk Eggs Other Dairy Spices and Sea...,lacto ovo vegetarian,vegetarian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,lacto-ovo-vegetarian
2,pandan chiffon cake,for 26 cents per serving this recipe covers ...,preheat the oven to 170c blend the pandan le...,all purpose flour; bay leaves; coconut milk; c...,Ethnic Foods Produce Spices and Seasonings Bev...,dairy free; lacto ovo vegetarian,vegetarian,NaN,NaN,dairy-free,NaN,NaN,NaN,NaN,dairy-free; lacto-ovo-vegetarian
3,pork chop with honey mustard and apples,pork chop with honey mustard and apples might...,pre heat your oven to 200c 400f line a roa...,apples; dijon mustard; garlic cloves; honey; j...,Meat Spices and Seasonings Condiments Oil Vine...,gluten free; dairy free; paleolithic; primal,NaN,NaN,gluten-free,dairy-free,NaN,NaN,NaN,NaN,gluten-free; dairy-free; paleolithic; primal
4,beet gnocchi with steak and brown butter sauce,the recipe beet gnocchi with steak and brown b...,cooking beets heat oven to 400 degrees wash be...,gnocchi; beets; olive oil; s p; goat cheese; r...,Produce Spices and Seasonings Meat Spices and ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Word2Vec

Here I use the Gensim library to create a word2vec embedding. 

Reference: 
- Gensim Word2Vec documentation: https://radimrehurek.com/gensim/models/word2vec.html
- DAS, P, 2020. How to train word2vec model using gensim library [online]. Medium. [viewed 26/03/2020]. Available from: https://medium.com/swlh/how-to-train-word2vec-model-using-gensim-library-115b35440c90

#### 2.1 Trainsform data into the required format

In [89]:
trainWord2Vec = False

In [90]:
if trainWord2Vec:
    download('stopwords')
    stop_words = stopwords.words('english')
    
    nltk.download('averaged_perceptron_tagger')
    nltk.download('wordnet')
    nltk.download('omw-1.4')

In [91]:
class Sentence(object):
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.n_rows = df.shape[0]
        self.lemmatizer = WordNetLemmatizer()
        
    def __iter__(self):
        for n in range(self.n_rows):
            row = self.df.iloc[n]
            sentence : List[str] = []
            for i, cell in enumerate(row):
                if type(cell) == str:
                    sent = cell.split("; ") if df.columns[i] in ["ingredients", "diets"] else cell.split()
                    sent = [w.lower() for w in sent if w not in stop_words]
                    if len(sent) > 0: sent[-1] = self.lemmatizer.lemmatize(sent[-1])
                    sentence += sent
            yield sentence

In [92]:
# initiase
if trainWord2Vec:
    sentences = Sentence(df)

### 2.2 Training

After experimenting with different values, the following setup seem to give reasonable results:
- vector size 300
- min count 1
- window 5
- epoch 30

Training time: ~80 seconds

In [93]:
if trainWord2Vec:
    start = time.time()
    model = Word2Vec(
        sentences, 
        min_count=1,
        vector_size=300, 
        workers=2,
        window=2,
        epochs=30, 
        sg=0
    )
    end = time.time()
    model.save("word2vec.model")
    print(f"Training comleted in {end-start}s.")

### 2.3 Generate ingredient suggestions

To get ingredient suggestions, we will provide the desired postiive (and maybe also negetive) keywords.

In [94]:
model = Word2Vec.load("word2vec.model")

In [95]:
model.wv.most_similar(positive=["vegan", "pesto", "pasta"], topn=20)

[('feta', 0.4106769561767578),
 ('pescatarian', 0.40386947989463806),
 ('alfredo', 0.39441919326782227),
 ('fresh mozzarella cheese', 0.3637278079986572),
 ('gorgonzola', 0.3603726327419281),
 ('arugula', 0.354171484708786),
 ('dressing', 0.3513053059577942),
 ('fetan', 0.34398210048675537),
 ('salad', 0.3417564332485199),
 ('dairy', 0.33915770053863525),
 ('goat’s', 0.3384309411048889),
 ('free', 0.3335084021091461),
 ('rigate', 0.3308608829975128),
 ('pappardelle', 0.3304375410079956),
 ('garlicky', 0.3240084946155548),
 ('orzo', 0.31973889470100403),
 ('pilaf', 0.3195817768573761),
 ('spaghetti', 0.31901875138282776),
 ('gluten-free', 0.31660717725753784),
 ('mozzarellan', 0.31027910113334656)]

The results are satisfactory. For example, even though "feta" and "fresh mozzarella cheese" are not vegan, they are somewhat resonable suggestions. There are at least no meat on the list. 

However, we also observed that there are a lot of non-ingredient words in the list of suggestions, such as "free". Therefore, we decided to find out to exclude suggestions that are not in the list of ingredients. 

### 2.4 Improved ingredient suggestion generator

Our strategy is:
1. Obtain a list of candicate incredients, save it in a list
2. Generate suggestions like above, but filter out suggestions that are not in the ingredient list

In [96]:
ingredients = df["ingredients"].to_list()
# ingredients_split = df["ingredient split"].to_list()

all_ingredients = []
for i in ingredients: 
    items = i.split("; ")
    all_ingredients += items

# for i in ingredients_split: 
#     items = i.split()
#     all_ingredients += items

all_ingredients = list(set(all_ingredients))

In [109]:
def get_ingredients(ingredients: List[str], 
                    recipe: List[str], 
                    positive: List[str], 
                    negative: List[str] = [], 
                    topn=20):
    
    pos = recipe.split() + positive*3
    candidates = model.wv.most_similar(positive=pos, negative=negative*2, topn=200)
    
    substitutions = []
    for ingredient_name, _ in candidates:
        if len(substitutions) >= topn:
            break
        if ingredient_name in ingredients:
            substitutions.append(ingredient_name)
    return substitutions

In [110]:
get_ingredients(all_ingredients, 
                recipe="pesto pasta", 
                positive=["vegan"], 
                negative=[], topn=20)

['gorgonzola',
 'feta',
 'crepes',
 'fresh mozzarella cheese',
 'arugula',
 'mushroom',
 'bulgur',
 'nutella',
 'mayo',
 'honey',
 'orzo',
 'baguettes',
 'buckwheat']

## 3. BERT

### 3.1 Using pretrained Bert model

There are a lot of powerful pretrained BERT models and we would like to see whether the expressive power of BERT would help it success this task better than Word2Vec.

In [111]:
from transformers import pipeline
import torch

In [131]:
pretrained_bert_pipeline = pipeline("fill-mask", 
                model="bert-base-uncased", 
                tokenizer="bert-base-uncased", 
                top_k=20)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [146]:
def get_ingredient_bert(pipeline, recipe):
    results = pipeline(recipe)
    return [''.join(r['token_str'].split()) for r in results]

In [148]:
mask_token = pretrained_bert_pipeline.tokenizer.mask_token
recipe = f"vegan pesto {mask_token} pasta"
get_ingredient_bert(pretrained_bert_pipeline, recipe)

['and',
 '&',
 'salad',
 'with',
 '-',
 ',',
 'fried',
 ':',
 'or',
 'chicken',
 '/',
 'homemade',
 'cheese',
 'fish',
 'rice',
 'style',
 'for',
 'sliced',
 'italian',
 'chile']

The results are not very satisfactory as none of the outputs seem to capture our requirements. The first ingredient output is chicken which is clearly not vegan and not even vegetarian. We would therefore like to explore whether custom trained BERT would perform better in this task. 

### 3.2 Custom - RoBerta model

There are multiple BERT-like models out there. We have chosen to train a RoBerta model, whose implementation is essentially the same with a few tweaks. In particular, it removes the next-sentence pretraining objective, and focus on the masked language modelling objective. This makes us believe that this is more suitable as we believe that the latter objective is more relevant to the task at hand. 

Reference:
- Roberta documentation [online]. Hugging Face. [viewed 30/03/2020]. Available from:. https://huggingface.co/docs/transformers/model_doc/roberta
- Briggs, J., How to Train a BERT Model From Scratch [online]. Medium. [viewed 30/30/2020]. Available from: https://towardsdatascience.com/how-to-train-a-bert-model-from-scratch-72cfce554fc6

Training time: 6 hours 18 mins

In [70]:
# Set train to True to train model 
trainBert = False

In [71]:
from tokenizers import ByteLevelBPETokenizer
from transformers import RobertaModel, RobertaTokenizerFast, RobertaConfig, RobertaForMaskedLM, AdamW
import os 
from tqdm.auto import tqdm
import glob

#### Create the text file for training

In [72]:
df1 = df.apply(lambda x: " ".join([cell for cell in x if type(cell)==str]), axis=1)

recipes = "\n".join(df1.to_list())
with open("recipes.txt", 'w', encoding="utf-8") as f:
    f.write(recipes)

#### Train tokenizer

In [115]:
tokenizer_path = "recipe_tokenizer"

In [116]:
if trainBert:
    tokenizer = ByteLevelBPETokenizer()
    
    # Train the tokenizer with text
    tokenizer.train(files=["recipes.txt"], 
                vocab_size=30_522, 
                min_frequency=1, 
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])
    os.mkdir("recipe_tokenizer")
    tokenizer.save_model(tokenizer_path)

In [117]:
tokenizer = RobertaTokenizerFast.from_pretrained(tokenizer_path)

In [118]:
tokenizer.save_pretrained(tokenizer_path)

('recipe_tokenizer/tokenizer_config.json',
 'recipe_tokenizer/special_tokens_map.json',
 'recipe_tokenizer/vocab.json',
 'recipe_tokenizer/merges.txt',
 'recipe_tokenizer/added_tokens.json',
 'recipe_tokenizer/tokenizer.json')

In [119]:
tokenizer("pesto pasta")

{'input_ids': [0, 8327, 974, 2], 'attention_mask': [1, 1, 1, 1]}

### Input pipeline

In [120]:
def mlm(tensor):
    """
    input: tensor of sentences
    output: tensor of sentences with masks words
    """
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4 # mask token
    return tensor

In [121]:
if trainBert:
    input_ids = []
    attn_mask = [] 
    labels = []

    with open("recipes.txt", 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')

    sample = tokenizer(lines, 
                       max_length=512, 
                       padding="max_length", 
                       truncation=True, 
                       return_tensors='pt')
    labels.append(sample.input_ids)
    attn_mask.append(sample.attention_mask)
    input_ids.append(mlm(sample.input_ids.detach().clone()))

In [122]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return self.encodings['input_ids'].shape[0]

    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [123]:
if trainBert:
    # set up for training
    input_ids = torch.cat(input_ids)
    attn_mask = torch.cat(attn_mask)
    labels = torch.cat(labels)
    
    encodings = {
    'input_ids': input_ids,
    'attention_mask': attn_mask,
    'labels': labels
    }
    
    dataset = Dataset(encodings)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)
    
    config = RobertaConfig(
    vocab_size = tokenizer.vocab_size,
    max_position_embeddings=514, 
    hidden_size = 768, 
    num_attention_heads=12, 
    num_hidden_layers= 6, 
    type_vocab_size = 1
    )

In [124]:
if trainBert:
    device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")
    optim = AdamW(model.parameters(), lr=1e-4)
    model.to(device)

In [125]:
if trainBert:
    loop = tqdm(dataloader, leave=True)

    for batch in loop:
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optim.step()

        loop.set_description(f"Epoch: {epochs}")
        loop.set_postfix(loss=loss.item())
    
    model.save_pretrained('./model')

#### Testing

In [126]:
model = RobertaModel.from_pretrained("./model")

Some weights of the model checkpoint at ./model were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./model and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [128]:
get_ingredient_custom_bert = pipeline("fill-mask", 
                model="model", 
                tokenizer="recipe_tokenizer", 
                top_k=20)

results = get_ingredient_custom_bert(f"vegan pesto {get_ingredient_custom_bert.tokenizer.mask_token} pasta")
results = [c['token_str'] for c in results]
results

[' ',
 ' and',
 ';',
 ' the',
 ' to',
 ' a',
 ' with',
 '<pad>',
 ' of',
 '  ',
 ' is',
 ' for',
 ' in',
 ' recipe',
 ' this',
 ' minutes',
 '-',
 'g',
 ' gluten',
 'free']